In [25]:
import tensorflow as tf
tf.__version__


'2.12.0'

In [41]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [42]:
subject = 'Parana - Consumo de Cimento (t)'
split_index = 6 #Referente aos 7 anos de input  

In [43]:
data = pd.read_csv('../Inputs Atuais/2014_01_model_input_PR.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data

,Ano,Paraná - PIB - Estadual,Paraná - PIB - Construção Civil,Paraná - PIB - Per Capita,Paraná - PIB - Preços de Mercado,Paraná - IDH,Paraná - IDH Renda,Paraná - Produção de Cimento (t),Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),...,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Paraná - Desemprego,Paraná - IDH Educacao,Parana - Consumo de Cimento (t)
0,2014,2.386532e+08,1.115283e+07,18.028052,1.924772e+08,0.785311,0.767235,5788.881494,5.676413e+08,2.779340e+09,...,20.725057,912.973266,1150.562966,18.275455,13.157699,74.605015,1.851460,7.462443,0.760152,4341.821857
1,2015,2.410111e+08,1.107686e+07,18.031270,1.926648e+08,0.788311,0.758000,5749.529291,5.179392e+08,2.561731e+09,...,22.054778,913.057535,1145.712487,18.177724,13.346348,75.170150,1.905966,7.813261,0.769526,4108.786286
2,2016,2.424341e+08,1.101317e+07,18.026717,1.926914e+08,0.788521,0.758363,5786.537903,4.524144e+08,2.516859e+09,...,22.315526,913.132268,1158.573328,18.115048,13.261122,74.043778,1.933375,7.840669,0.771574,3869.334857
3,2017,2.428100e+08,1.097443e+07,18.022550,1.926406e+08,0.787750,0.758578,5902.212019,4.213689e+08,2.497710e+09,...,21.977949,913.165670,1160.957791,18.281050,13.470416,73.546507,1.889856,7.553100,0.771445,3764.518000
4,2018,2.423355e+08,1.097890e+07,18.011594,1.924273e+08,0.786909,0.758540,6085.585319,4.166580e+08,2.797145e+09,...,21.569960,913.197376,1181.996564,18.350169,13.529500,73.356104,1.829790,7.440077,0.772121,3773.354000
5,2019,2.408185e+08,1.099642e+07,17.986197,1.920808e+08,0.785264,0.757862,6533.257168,4.462625e+08,3.484252e+09,...,21.479228,913.132269,1192.379390,18.270645,13.605491,72.531518,1.824138,7.368470,0.771678,3786.974000
6,2020,2.378722e+08,1.102014e+07,17.944016,1.914461e+08,0.783289,0.757303,7078.134333,5.489412e+08,4.114911e+09,...,21.933567,912.972214,1166.191854,18.118751,13.563808,69.481534,1.794314,7.229325,0.771012,4235.532000
7,2021,2.333756e+08,1.101048e+07,17.909608,1.907484e+08,0.782951,0.757877,7344.760727,6.638811e+08,4.421848e+09,...,21.975729,912.872647,1140.544113,17.846885,13.396376,68.473686,1.778305,7.094891,0.769195,4784.413000
8,2022,2.279306e+08,1.100628e+07,17.865923,1.898715e+08,0.784219,0.759138,7392.021558,7.010332e+08,4.533216e+09,...,21.803048,912.860902,1136.860026,17.676137,13.339456,70.038046,1.798074,7.116836,0.768213,4538.256000


In [44]:
input_data = data.iloc[:-2, 1:-1]
input_data

,Paraná - PIB - Estadual,Paraná - PIB - Construção Civil,Paraná - PIB - Per Capita,Paraná - PIB - Preços de Mercado,Paraná - IDH,Paraná - IDH Renda,Paraná - Produção de Cimento (t),Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Paraná - IDH Longevidade,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Paraná - Desemprego,Paraná - IDH Educacao
0,2.386532e+08,1.115283e+07,18.028052,1.924772e+08,0.785311,0.767235,5788.881494,5.676413e+08,2.779340e+09,1.954361e+10,...,0.831380,20.725057,912.973266,1150.562966,18.275455,13.157699,74.605015,1.851460,7.462443,0.760152
1,2.410111e+08,1.107686e+07,18.031270,1.926648e+08,0.788311,0.758000,5749.529291,5.179392e+08,2.561731e+09,1.327596e+10,...,0.840756,22.054778,913.057535,1145.712487,18.177724,13.346348,75.170150,1.905966,7.813261,0.769526
2,2.424341e+08,1.101317e+07,18.026717,1.926914e+08,0.788521,0.758363,5786.537903,4.524144e+08,2.516859e+09,1.011370e+10,...,0.838825,22.315526,913.132268,1158.573328,18.115048,13.261122,74.043778,1.933375,7.840669,0.771574
3,2.428100e+08,1.097443e+07,18.022550,1.926406e+08,0.787750,0.758578,5902.212019,4.213689e+08,2.497710e+09,8.456309e+09,...,0.836263,21.977949,913.165670,1160.957791,18.281050,13.470416,73.546507,1.889856,7.553100,0.771445
4,2.423355e+08,1.097890e+07,18.011594,1.924273e+08,0.786909,0.758540,6085.585319,4.166580e+08,2.797145e+09,7.937760e+09,...,0.833015,21.569960,913.197376,1181.996564,18.350169,13.529500,73.356104,1.829790,7.440077,0.772121
5,2.408185e+08,1.099642e+07,17.986197,1.920808e+08,0.785264,0.757862,6533.257168,4.462625e+08,3.484252e+09,1.091282e+10,...,0.829016,21.479228,913.132269,1192.379390,18.270645,13.605491,72.531518,1.824138,7.368470,0.771678
6,2.378722e+08,1.102014e+07,17.944016,1.914461e+08,0.783289,0.757303,7078.134333,5.489412e+08,4.114911e+09,1.625333e+10,...,0.824199,21.933567,912.972214,1166.191854,18.118751,13.563808,69.481534,1.794314,7.229325,0.771012


In [45]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0    4108.786286
1    3869.334857
2    3764.518000
3    3773.354000
4    3786.974000
5    4235.532000
6    4784.413000
7    4538.256000
8            NaN
Name: Parana - Consumo de Cimento (t), dtype: float64

In [46]:
# input para treinamento
train_input = (input_data.iloc[:-1] - np.mean(input_data.iloc[:-1], axis=0)) / np.std(input_data.iloc[:-1], axis=0)
train_input 

,Paraná - PIB - Estadual,Paraná - PIB - Construção Civil,Paraná - PIB - Per Capita,Paraná - PIB - Preços de Mercado,Paraná - IDH,Paraná - IDH Renda,Paraná - Produção de Cimento (t),Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Paraná - IDH Longevidade,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Paraná - Desemprego,Paraná - IDH Educacao
0,-1.906656,1.894477,0.669569,-0.094438,-1.287304,2.229154,-0.677048,1.779974,0.019134,2.003355,...,-0.847280,-1.864612,-1.835260,-0.865476,0.396527,-1.528752,0.849857,-0.520677,-0.640122,-2.193310
1,-0.235693,0.702329,0.878332,0.798411,0.984100,-0.526020,-0.820715,0.870373,-0.621387,0.401153,...,1.425338,0.712671,-0.701961,-1.155642,-0.876568,-0.313917,1.508230,0.832647,1.275529,0.026107
2,0.772650,-0.297109,0.582975,0.925018,1.143151,-0.417550,-0.685604,-0.328802,-0.753466,-0.407217,...,0.957204,1.218057,0.303093,-0.386278,-1.693009,-0.862748,0.196027,1.513189,1.425194,0.510954
3,1.039057,-0.905049,0.312693,0.683077,0.559532,-0.353458,-0.263302,-0.896969,-0.809829,-0.830898,...,0.336332,0.563761,0.752309,-0.243634,0.469400,0.485035,-0.383285,0.432639,-0.145086,0.480264
4,0.702826,-0.834767,-0.398055,-0.331855,-0.077143,-0.364901,0.406156,-0.983185,0.071542,-0.963455,...,-0.451164,-0.227010,1.178714,1.014953,1.369786,0.865514,-0.605101,-1.058722,-0.762251,0.640497
5,-0.372184,-0.559880,-2.045514,-1.980213,-1.322337,-0.567224,2.040512,-0.441390,2.094006,-0.202939,...,-1.420430,-0.402867,0.303104,1.636077,0.333865,1.354868,-1.565728,-1.199076,-1.153263,0.535489


In [48]:
# Alvo para treinamento
train_target = target_data.iloc[:split_index]
train_target

0    4108.786286
1    3869.334857
2    3764.518000
3    3773.354000
4    3786.974000
5    4235.532000
Name: Parana - Consumo de Cimento (t), dtype: float64

In [49]:
#input de test (Ano 2021)
test_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[-1:]
test_input

,Paraná - PIB - Estadual,Paraná - PIB - Construção Civil,Paraná - PIB - Per Capita,Paraná - PIB - Preços de Mercado,Paraná - IDH,Paraná - IDH Renda,Paraná - Produção de Cimento (t),Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Paraná - IDH Longevidade,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Paraná - Desemprego,Paraná - IDH Educacao
6,-1.66797,-0.173298,-2.143284,-2.165084,-1.785883,-0.654572,2.047604,1.16956,2.035192,0.985175,...,-1.712832,0.435262,-1.40336,0.064303,-1.293233,0.93046,-2.176019,-1.44822,-1.435264,0.346341


In [50]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[split_index:-2]
test_target

6    4784.413
Name: Parana - Consumo de Cimento (t), dtype: float64

In [54]:
# Rede neural feed-forward com optmizador Estocástico
def neural_network_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=1000, 
                                                      verbose=1, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(74, activation='tanh', input_shape=(train_input.shape[1],)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(296, activation='tanh'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(148, activation='tanh'),
        tf.keras.layers.Dense(37, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train_input.iloc[:-1], 
                        train_target.iloc[:-1], 
                        epochs=5000, 
                        batch_size=len(train_input), 
                        validation_data=(train_input.iloc[-1:], train_target.iloc[-1:]),
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [55]:
def get_a_good_seed(train_input, train_target, test_target, test_input):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(100)]
    print(random_seeds)

    best_val_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = neural_network_model(train_input, train_target, 0, seed)

        current_val_loss = min(history.history['val_loss'])
        print(f"val_loss: {current_val_loss}")

        if current_val_loss < best_val_loss:
            best_val_loss = current_val_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")

    return winner_seed

In [56]:
winner_seed = get_a_good_seed(train_input, train_target, test_target, test_input)
print(f"\n\nfinal_seed: {winner_seed}")

[3059518028, 917447096, 870628346, 915772297, 778533778, 1206497292, 2403935654, 3030069690, 2666640515, 2241126650, 2050546614, 798039752, 4059285304, 3957263115, 706994864, 3247311909, 31593958, 92831467, 617156287, 3079924325, 195459138, 3293033199, 791167706, 2545084998, 3566071957, 1501190685, 219870010, 3034669965, 2810099980, 168911602, 757288022, 669148571, 4282733003, 2190423753, 1888952750, 2566317157, 3838357431, 2899192741, 1896991233, 1489701019, 2207293125, 3954181999, 3695161779, 742908751, 2969060706, 3313208704, 3390581937, 2496827851, 3202112473, 842404772, 2032464146, 1007056708, 1775360765, 730008760, 3515743923, 3636767923, 4294134822, 1209368449, 3047973098, 2658640646, 499226616, 1943338157, 3467060559, 1300150570, 1551267216, 689557921, 138274747, 4271725154, 3300759931, 3336083360, 3997268973, 2833120971, 246455187, 1528732157, 2250495122, 3251562194, 2158187904, 3687180631, 1337262742, 1366826301, 2154215160, 3813759558, 3914412890, 579530091, 994114054, 27846

Restoring model weights from the end of the best epoch: 1358.
Epoch 2358: early stopping
val_loss: 81485.0234375


Step: 42 ___________________________________________
Restoring model weights from the end of the best epoch: 1014.
Epoch 2014: early stopping
val_loss: 82839.265625


Step: 43 ___________________________________________
Restoring model weights from the end of the best epoch: 1628.
Epoch 2628: early stopping
val_loss: 63878.48828125


Step: 44 ___________________________________________
Restoring model weights from the end of the best epoch: 503.
Epoch 1503: early stopping
val_loss: 8776.06640625


Step: 45 ___________________________________________
Restoring model weights from the end of the best epoch: 2470.
Epoch 3470: early stopping
val_loss: 66590.4609375


Step: 46 ___________________________________________
Restoring model weights from the end of the best epoch: 1303.
Epoch 2303: early stopping
val_loss: 89378.2265625


Step: 47 _____________________________________

Restoring model weights from the end of the best epoch: 1991.
Epoch 2991: early stopping
val_loss: 80410.875


Step: 91 ___________________________________________
Restoring model weights from the end of the best epoch: 794.
Epoch 1794: early stopping
val_loss: 42088.28515625


Step: 92 ___________________________________________
Restoring model weights from the end of the best epoch: 574.
Epoch 1574: early stopping
val_loss: 38242.78125


Step: 93 ___________________________________________
Restoring model weights from the end of the best epoch: 952.
Epoch 1952: early stopping
val_loss: 106.79122924804688


Step: 94 ___________________________________________
Restoring model weights from the end of the best epoch: 1033.
Epoch 2033: early stopping
val_loss: 61369.17578125


Step: 95 ___________________________________________
Restoring model weights from the end of the best epoch: 1095.
Epoch 2095: early stopping
val_loss: 65812.7890625


Step: 96 ______________________________________

In [57]:
trained_model, history = neural_network_model(train_input, train_target, want_verbose=1, seed=winner_seed)

Epoch 1/5000
1/1 [==============================] - 0s 300ms/step - loss: 14921710.0000 - val_loss: 17971476.0000
Epoch 2/5000
1/1 [==============================] - 0s 12ms/step - loss: 14769632.0000 - val_loss: 16913114.0000
Epoch 3/5000
1/1 [==============================] - 0s 11ms/step - loss: 15452744.0000 - val_loss: 14926134.0000
Epoch 4/5000
1/1 [==============================] - 0s 11ms/step - loss: 14373322.0000 - val_loss: 16628294.0000
Epoch 5/5000
1/1 [==============================] - 0s 11ms/step - loss: 13767165.0000 - val_loss: 17131980.0000
Epoch 6/5000
1/1 [==============================] - 0s 11ms/step - loss: 13643896.0000 - val_loss: 19719428.0000
Epoch 7/5000
1/1 [==============================] - 0s 12ms/step - loss: 13092421.0000 - val_loss: 14037144.0000
Epoch 8/5000
1/1 [==============================] - 0s 11ms/step - loss: 12151685.0000 - val_loss: 12820008.0000
Epoch 9/5000
1/1 [==============================] - 0s 12ms/step - loss: 11184675.0000 - val_lo

1/1 [==============================] - 0s 11ms/step - loss: 2090736.2500 - val_loss: 19130654.0000
Epoch 74/5000
1/1 [==============================] - 0s 11ms/step - loss: 3160345.5000 - val_loss: 19039714.0000
Epoch 75/5000
1/1 [==============================] - 0s 12ms/step - loss: 3360682.0000 - val_loss: 19012100.0000
Epoch 76/5000
1/1 [==============================] - 0s 11ms/step - loss: 2554622.7500 - val_loss: 18837580.0000
Epoch 77/5000
1/1 [==============================] - 0s 11ms/step - loss: 2987446.7500 - val_loss: 18765878.0000
Epoch 78/5000
1/1 [==============================] - 0s 12ms/step - loss: 3528629.5000 - val_loss: 18403706.0000
Epoch 79/5000
1/1 [==============================] - 0s 12ms/step - loss: 3022175.7500 - val_loss: 18600192.0000
Epoch 80/5000
1/1 [==============================] - 0s 12ms/step - loss: 2713925.2500 - val_loss: 15827876.0000
Epoch 81/5000
1/1 [==============================] - 0s 11ms/step - loss: 3629932.5000 - val_loss: 15252412.00

1/1 [==============================] - 0s 12ms/step - loss: 946547.3125 - val_loss: 8001923.0000
Epoch 146/5000
1/1 [==============================] - 0s 12ms/step - loss: 1090728.6250 - val_loss: 7897508.0000
Epoch 147/5000
1/1 [==============================] - 0s 12ms/step - loss: 1098049.5000 - val_loss: 7879410.5000
Epoch 148/5000
1/1 [==============================] - 0s 12ms/step - loss: 908881.3750 - val_loss: 7947284.0000
Epoch 149/5000
1/1 [==============================] - 0s 12ms/step - loss: 1049812.5000 - val_loss: 7896053.5000
Epoch 150/5000
1/1 [==============================] - 0s 11ms/step - loss: 987063.3125 - val_loss: 7696723.0000
Epoch 151/5000
1/1 [==============================] - 0s 11ms/step - loss: 823297.5000 - val_loss: 7776380.5000
Epoch 152/5000
1/1 [==============================] - 0s 12ms/step - loss: 947526.5000 - val_loss: 7514454.0000
Epoch 153/5000
1/1 [==============================] - 0s 11ms/step - loss: 1391055.3750 - val_loss: 7433477.5000
Epo

1/1 [==============================] - 0s 12ms/step - loss: 662281.3750 - val_loss: 3536642.0000
Epoch 219/5000
1/1 [==============================] - 0s 12ms/step - loss: 338855.1875 - val_loss: 3450688.0000
Epoch 220/5000
1/1 [==============================] - 0s 12ms/step - loss: 462944.5000 - val_loss: 3466409.5000
Epoch 221/5000
1/1 [==============================] - 0s 11ms/step - loss: 395150.3438 - val_loss: 3448215.0000
Epoch 222/5000
1/1 [==============================] - 0s 11ms/step - loss: 341080.3750 - val_loss: 3468604.2500
Epoch 223/5000
1/1 [==============================] - 0s 11ms/step - loss: 303853.8125 - val_loss: 3388845.0000
Epoch 224/5000
1/1 [==============================] - 0s 11ms/step - loss: 543749.5000 - val_loss: 3342353.5000
Epoch 225/5000
1/1 [==============================] - 0s 11ms/step - loss: 328396.6875 - val_loss: 3401733.5000
Epoch 226/5000
1/1 [==============================] - 0s 11ms/step - loss: 280053.5625 - val_loss: 3264742.5000
Epoch 2

1/1 [==============================] - 0s 11ms/step - loss: 253880.7500 - val_loss: 1839045.8750
Epoch 292/5000
1/1 [==============================] - 0s 11ms/step - loss: 131037.7031 - val_loss: 1765597.3750
Epoch 293/5000
1/1 [==============================] - 0s 11ms/step - loss: 196065.4219 - val_loss: 1736233.3750
Epoch 294/5000
1/1 [==============================] - 0s 11ms/step - loss: 129390.4531 - val_loss: 1737457.3750
Epoch 295/5000
1/1 [==============================] - 0s 11ms/step - loss: 255371.7969 - val_loss: 1742926.0000
Epoch 296/5000
1/1 [==============================] - 0s 11ms/step - loss: 193527.8281 - val_loss: 1638892.5000
Epoch 297/5000
1/1 [==============================] - 0s 12ms/step - loss: 160945.6250 - val_loss: 1645249.6250
Epoch 298/5000
1/1 [==============================] - 0s 14ms/step - loss: 127571.3516 - val_loss: 1651066.8750
Epoch 299/5000
1/1 [==============================] - 0s 12ms/step - loss: 138748.3281 - val_loss: 1676420.5000
Epoch 3

...

Epoch 1423/5000
1/1 [==============================] - 0s 12ms/step - loss: 22926.2559 - val_loss: 48615.6211
Epoch 1424/5000
1/1 [==============================] - 0s 13ms/step - loss: 56520.0000 - val_loss: 32480.0293
Epoch 1425/5000
1/1 [==============================] - 0s 12ms/step - loss: 52848.7383 - val_loss: 39372.7891
Epoch 1426/5000
1/1 [==============================] - 0s 12ms/step - loss: 52296.7500 - val_loss: 35986.1641
Epoch 1427/5000
1/1 [==============================] - 0s 13ms/step - loss: 71751.7188 - val_loss: 32617.3652
Epoch 1428/5000
1/1 [==============================] - 0s 13ms/step - loss: 44707.6641 - val_loss: 28498.0723
Epoch 1429/5000
1/1 [==============================] - 0s 12ms/step - loss: 76193.7031 - val_loss: 28655.8945
Epoch 1430/5000
1/1 [==============================] - 0s 14ms/step - loss: 57138.1992 - val_loss: 29858.8438
Epoch 1431/5000
1/1 [==============================] - 0s 12ms/step - loss: 73137.2734 - val_loss: 38289.6797
Epoch 1432

1/1 [==============================] - 0s 12ms/step - loss: 30198.5410 - val_loss: 21784.0586
Epoch 1498/5000
1/1 [==============================] - 0s 12ms/step - loss: 23437.8613 - val_loss: 17300.8555
Epoch 1499/5000
1/1 [==============================] - 0s 12ms/step - loss: 3300.8286 - val_loss: 18801.3906
Epoch 1500/5000
1/1 [==============================] - 0s 12ms/step - loss: 38180.8125 - val_loss: 21228.8320
Epoch 1501/5000
1/1 [==============================] - 0s 12ms/step - loss: 39563.1523 - val_loss: 20531.9648
Epoch 1502/5000
1/1 [==============================] - 0s 11ms/step - loss: 12162.9834 - val_loss: 15139.4521
Epoch 1503/5000
1/1 [==============================] - 0s 11ms/step - loss: 37642.6758 - val_loss: 20283.5039
Epoch 1504/5000
1/1 [==============================] - 0s 11ms/step - loss: 14494.8926 - val_loss: 17498.5879
Epoch 1505/5000
1/1 [==============================] - 0s 11ms/step - loss: 36739.4023 - val_loss: 23689.3125
Epoch 1506/5000
1/1 [======

Epoch 1572/5000
1/1 [==============================] - 0s 13ms/step - loss: 24300.1816 - val_loss: 23201.5527
Epoch 1573/5000
1/1 [==============================] - 0s 13ms/step - loss: 21413.2910 - val_loss: 28786.6406
Epoch 1574/5000
1/1 [==============================] - 0s 12ms/step - loss: 28163.6875 - val_loss: 24831.6348
Epoch 1575/5000
1/1 [==============================] - 0s 12ms/step - loss: 23086.0566 - val_loss: 23809.9375
Epoch 1576/5000
1/1 [==============================] - 0s 12ms/step - loss: 20389.2520 - val_loss: 26387.8457
Epoch 1577/5000
1/1 [==============================] - 0s 12ms/step - loss: 6780.7764 - val_loss: 28107.7988
Epoch 1578/5000
1/1 [==============================] - 0s 12ms/step - loss: 13098.0439 - val_loss: 26641.6348
Epoch 1579/5000
1/1 [==============================] - 0s 12ms/step - loss: 2281.2778 - val_loss: 26146.2422
Epoch 1580/5000
1/1 [==============================] - 0s 12ms/step - loss: 11869.5488 - val_loss: 20883.1426
Epoch 1581/5

1/1 [==============================] - 0s 12ms/step - loss: 27709.9531 - val_loss: 27472.9004
Epoch 1647/5000
1/1 [==============================] - 0s 12ms/step - loss: 17663.6465 - val_loss: 27808.9824
Epoch 1648/5000
1/1 [==============================] - 0s 12ms/step - loss: 8375.0996 - val_loss: 30006.0879
Epoch 1649/5000
1/1 [==============================] - 0s 11ms/step - loss: 41355.1055 - val_loss: 35603.3398
Epoch 1650/5000
1/1 [==============================] - 0s 12ms/step - loss: 17650.8594 - val_loss: 42344.7188
Epoch 1651/5000
1/1 [==============================] - 0s 12ms/step - loss: 9643.2217 - val_loss: 44633.6797
Epoch 1652/5000
1/1 [==============================] - 0s 12ms/step - loss: 19530.1973 - val_loss: 37481.4688
Epoch 1653/5000
1/1 [==============================] - 0s 12ms/step - loss: 12049.6533 - val_loss: 39326.7812
Epoch 1654/5000
1/1 [==============================] - 0s 12ms/step - loss: 15765.9248 - val_loss: 39285.1562
Epoch 1655/5000
1/1 [=======

Epoch 1721/5000
1/1 [==============================] - 0s 12ms/step - loss: 1976.3137 - val_loss: 37677.5000
Epoch 1722/5000
1/1 [==============================] - 0s 12ms/step - loss: 8872.7080 - val_loss: 36556.7695
Epoch 1723/5000
1/1 [==============================] - 0s 12ms/step - loss: 6070.0947 - val_loss: 36360.5117
Epoch 1724/5000
1/1 [==============================] - 0s 11ms/step - loss: 12791.7393 - val_loss: 33559.3594
Epoch 1725/5000
1/1 [==============================] - 0s 11ms/step - loss: 31539.5586 - val_loss: 35425.4727
Epoch 1726/5000
1/1 [==============================] - 0s 11ms/step - loss: 13843.3125 - val_loss: 33698.6875
Epoch 1727/5000
1/1 [==============================] - 0s 12ms/step - loss: 6671.1885 - val_loss: 32098.0117
Epoch 1728/5000
1/1 [==============================] - 0s 12ms/step - loss: 6825.3857 - val_loss: 33823.1250
Epoch 1729/5000
1/1 [==============================] - 0s 11ms/step - loss: 2566.5166 - val_loss: 33161.5117
Epoch 1730/5000


In [58]:
prediction = trained_model.predict(test_input)
print(f"{test_target.values}(test_target) - {prediction}(prediction) = {mean_absolute_error(test_target, prediction)}")

1/1 [==============================] - 0s 44ms/step
[4784.413](test_target) - [[4136.4414]](prediction) = 647.9715937499996


In [59]:
def model_time_series_mae_tester(data, window, seed):
    if window > 2:

        # Copia e organiza data geral
        predictions = []
        targets = []
        data_copy = data.copy()
        data_copy[subject] = data[subject].shift(-1)
        data_copy = data_copy.iloc[:-1,1:]
        
        input_df = data_copy.iloc[:, :-1]
        target_df = data_copy.iloc[:, -1:]

        for i in range(window, len(data_copy) + 1):

            train_input = input_df.iloc[:i - 1]
            train_input = (train_input - np.mean(train_input, axis=0)) / np.std(train_input, axis=0)
            print("train_input:")
            display(train_input)

            train_target = target_df.iloc[:i - 1]
            print("train_target:")
            display(train_target)

            test_input = input_df.iloc[:i]
            test_input = ((test_input - np.mean(test_input, axis=0)) / np.std(test_input, axis=0)).iloc[-1:]
            print("test_input:")
            display(test_input)

            test_target = target_df.iloc[i - 1:i]
            print("test_target:")
            display(test_target)
            
            model, hitory = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)

            prediction = model.predict(test_input)

            current_error = mean_absolute_error(test_target, prediction)
            print(f"Error: {current_error}\n\n")

            predictions.append(prediction[0].item())
            targets.append(test_target.iloc[0, 0])

        return predictions, targets

In [60]:
predictions, targets = model_time_series_mae_tester(data.iloc[:-1,:], 4, winner_seed)
predictions

train_input:


,Paraná - PIB - Estadual,Paraná - PIB - Construção Civil,Paraná - PIB - Per Capita,Paraná - PIB - Preços de Mercado,Paraná - IDH,Paraná - IDH Renda,Paraná - Produção de Cimento (t),Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Paraná - IDH Longevidade,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Paraná - Desemprego,Paraná - IDH Educacao
0,-1.312398,1.259022,-0.328493,-1.405006,-1.411792,1.413359,0.771123,1.165029,1.396038,1.335266,...,-1.387050,-1.397598,-1.248481,-0.198623,1.303629,-1.262166,-0.002826,-1.335649,-1.411225,-1.394082
1,0.199900,-0.071704,1.355493,0.562973,0.634261,-0.749250,-1.412220,0.111769,-0.502296,-0.264151,...,0.932417,0.511608,0.048939,-1.113294,-0.177036,1.183535,1.226155,0.265293,0.626030,0.491123
2,1.112498,-1.187318,-1.027000,0.842033,0.777532,-0.664109,0.641096,-1.276798,-0.893743,-1.071116,...,0.454633,0.885990,1.199542,1.311917,-1.126592,0.078631,-1.223329,1.070356,0.785195,0.902959


train_target:


,Parana - Consumo de Cimento (t)
0,4108.786286
1,3869.334857
2,3764.518000


test_input:


...

,Paraná - PIB - Estadual,Paraná - PIB - Construção Civil,Paraná - PIB - Per Capita,Paraná - PIB - Preços de Mercado,Paraná - IDH,Paraná - IDH Renda,Paraná - Produção de Cimento (t),Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Paraná - IDH Longevidade,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Paraná - Desemprego,Paraná - IDH Educacao
6,-1.66797,-0.173298,-2.143284,-2.165084,-1.785883,-0.654572,2.047604,1.16956,2.035192,0.985175,...,-1.712832,0.435262,-1.40336,0.064303,-1.293233,0.93046,-2.176019,-1.44822,-1.435264,0.346341


test_target:


,Parana - Consumo de Cimento (t)
6,4784.413


Restoring model weights from the end of the best epoch: 762.
Epoch 1762: early stopping
1/1 [==============================] - 0s 34ms/step
Error: 647.9715937499996




[4044.272216796875, 4068.615478515625, 3787.157470703125, 4136.44140625]

In [61]:
display(targets)
display(predictions)

[3773.354, 3786.974, 4235.532, 4784.413]

[4044.272216796875, 4068.615478515625, 3787.157470703125, 4136.44140625]

In [62]:
mae = mean_absolute_error(predictions, targets)
mae

412.2264545898437

In [63]:
porcentage = mae/np.mean(targets)
porcentage

0.0994498593816504